# Atom Lattice Analysis - Demonstration Notebook

## Section 1: Introduction

This notebook demonstrates the workflow for analyzing atom lattice structures using the tools provided in this project. 
It covers:
- Loading and preprocessing microscopy images.
- Analyzing lattice and sublattice structures.
- Visualizing displacement vectors.
- Performing clustering analysis on displacement data.

## Section 2: Setup

In [ ]:
# Import Required Libraries
import os
import numpy as np
import matplotlib.pyplot as plt

# Import custom modules
from src.image_handler import ImageHandler
from src.image_processor import ImageProcessor
from src.atom_lattice_analyzer import AtomLatticeAnalyzer
from src.displacement_analyzer import DisplacementAnalyzer
from src.clustering import ClusterAnalyzer

### Define Paths and Parameters

In [ ]:
# Paths
project_dir = os.getcwd()
images_dir = os.path.join(project_dir, "images")
results_dir = os.path.join(project_dir, "results")

# Create results directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

## Section 3: Data Loading

In [ ]:
# Load Images
handler = ImageHandler(images_dir)
print("Available images:", handler.images)

# Load the first image
image_name = handler.images[0]
image = handler.load_image(image_name)
image.plot()


## Section 4: Image Processing

In [ ]:
# Rotate and Crop the Image
processor = ImageProcessor(image)

# Rotate image
rotation_angle = -3.5  # degrees
processor.rotate(rotation_angle)
processor.plot()

# Crop image interactively
cropped_image = processor.crop()
cropped_image.plot()

## Section 5: Lattice Analysis

In [ ]:
# Detect and Refine Sublattices
analyzer = AtomLatticeAnalyzer(cropped_image)

# Detect Ba sublattice
sublatticeBa = analyzer.find_sublattice(separation_range=(15, 19), separation=5, color='red')
sublatticeBa.plot()

# Refine Ba sublattice
analyzer.refine_sublattice_positions(sublatticeBa, percent_to_nn=0.25)
analyzer.plot_sublattice(sublatticeBa)

# Construct zone axes for Ba
zone_axes = analyzer.construct_zone_axes(sublatticeBa)
print(f"Zone axes for Ba: {zone_axes}")

# ### Detect Secondary Sublattice
selected_zone_axis = zone_axes[2]  # Choose a specific zone axis
print(f"Selected zone axis for Ti: {selected_zone_axis}")

# Find Ti sublattice positions
atom_positions_Ti, image_atoms_subtracted = analyzer.find_second_sublattice(sublatticeBa, selected_zone_axis)

# Create Ti sublattice
sublatticeTi = analyzer.find_sublattice(
    separation_range=(15, 19), separation=5, color='blue'
)
analyzer.refine_sublattice_positions(sublatticeTi, percent_to_nn=0.24)
analyzer.plot_sublattice(sublatticeTi)

## Section 6: Displacement Analysis

In [ ]:
# Calculate Displacement Vectors
sampling = cropped_image.axes_manager[-1].scale
unit = cropped_image.axes_manager[-1].units

displacement_analyzer = DisplacementAnalyzer(
    sublatticeBa, sublatticeTi, cropped_image, sampling, unit, results_dir, "BaTiO3_analysis"
)

# Compute displacement vectors
z1, z2 = zone_axes[1], zone_axes[2]
x, y, u, v = displacement_analyzer.calculate_displacement(z1, z2)

# ### Visualize Displacement Vectors
# Plot vector magnitude
displacement_analyzer.plot_magnitude_overlay(x, y, u, v)

# Plot angle distribution
displacement_analyzer.plot_angle_overlay(x, y, u, v)

## Section 7: Clustering

In [ ]:
# Prepare Data for Clustering
original_data = np.array([x, y, u, v]).T

# ### Perform Clustering
cluster_analyzer = ClusterAnalyzer(original_data, number_of_cores=4)

# KMeans clustering
n_clusters = 4
cluster_labels = cluster_analyzer.kmeans(n_clusters=n_clusters)

# Visualize clusters
plt.figure(figsize=(8, 6))
plt.scatter(original_data[:, 0], original_data[:, 1], c=cluster_labels, cmap='viridis', marker='o')
plt.title("KMeans Clustering on Displacement Data")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

## Section 8: Results

In [ ]:
# Save Results
print(f"Results saved in: {results_dir}")